In [3]:
import subprocess
import os
import hashlib
import datetime
import json
import shutil

# 환경 변수 세팅
ADB_PATH = r"E:\platform-tools-latest-windows\platform-tools\adb.exe"  # ADB 전체 경로 지정
ANDROID_IMG_REMOTE_PATH = "/data/user/0/com.android.example/..."  # 실제 이미지 경로로 수정
LOCAL_ENCRYPTED_IMG = "userdata-qemu.img.qcow2"
LOCAL_DECRYPTED_IMG = "userdata-decrypted.img"
FBE_DECRYPT_SCRIPT = "fbe-decrypt.mjs"  # Node.js 복호화 스크립트명
OUTPUT_DIR = "artifact_backup"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def run_adb(command):
    try:
        # command에서 'adb'를 전체 경로로 교체
        if isinstance(command, str):
            command = command.replace('adb', ADB_PATH, 1)
        elif isinstance(command, list):
            command[0] = ADB_PATH
            
        result = subprocess.run(command, shell=True, text=True, capture_output=True, timeout=60)
        if result.returncode != 0:
            print(f"Error running '{command}': {result.stderr}")
            return None
        return result.stdout
    except Exception as e:
        print(f"Exception running '{command}': {e}")
        return None

def timestamp():
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

def save_text(filename, text):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)

def adb_pull_image():
    print("[0] 암호화 이미지 ADB로 추출 중...")
    # 이미지 파일 기존 삭제
    if os.path.exists(LOCAL_ENCRYPTED_IMG):
        os.remove(LOCAL_ENCRYPTED_IMG)
    result = subprocess.run(
        [ADB_PATH, "pull", ANDROID_IMG_REMOTE_PATH, LOCAL_ENCRYPTED_IMG],  # ADB_PATH 사용
        capture_output=True, text=True
    )
    if result.returncode != 0:
        print("ADB pull 실패:", result.stderr)
        return False
    print("이미지 추출 완료:", LOCAL_ENCRYPTED_IMG)
    return True

def decrypt_image_by_node():
    print("[1] Node.js로 이미지 복호화 중...")
    if os.path.exists(LOCAL_DECRYPTED_IMG):
        os.remove(LOCAL_DECRYPTED_IMG)
    # 옵션과 파일명은 사용하시는 mjs 스크립트의 실제 usage에 맞게 수정
    args = ["node", FBE_DECRYPT_SCRIPT, LOCAL_ENCRYPTED_IMG, LOCAL_DECRYPTED_IMG]
    result = subprocess.run(args, capture_output=True, text=True)
    print("Node.js 표준 출력:", result.stdout)
    if result.returncode != 0 or not os.path.exists(LOCAL_DECRYPTED_IMG):
        print("복호화 실패:", result.stderr)
        return False
    print("복호화 성공:", LOCAL_DECRYPTED_IMG)
    return True

def mount_image_and_extract_apps(mount_point="/mnt/wearos_img"):
    # root 권한 필요, 로컬 Linux 환경 예시 (Windows는 별도 이미지 탐색/추출 필요)
    print("[2] 복호화 이미지 마운트 및 앱 리스트 추출 ...")
    if not os.path.isdir(mount_point):
        os.makedirs(mount_point)
    # 마운트 해제 후 재마운트 (실무에서는 에러처리 필수)
    subprocess.run(f"sudo umount {mount_point}", shell=True)
    result = subprocess.run(
        f"sudo mount -o loop {LOCAL_DECRYPTED_IMG} {mount_point}", shell=True
    )
    if result.returncode != 0:
        print("이미지 마운트 실패")
        return []
    app_base = os.path.join(mount_point, "data", "data")
    if not os.path.exists(app_base):
        print("앱 데이터 디렉토리 없음")
        return []
    app_pkgs = [
        d for d in os.listdir(app_base)
        if d.startswith("com.") or d.startswith("kr.")
    ]
    print(f"탐색된 앱: {app_pkgs}")
    return [os.path.join(app_base, x) for x in app_pkgs]

def extract_artifacts_from_path(app_path):
    app_name = os.path.basename(app_path)
    out_dir = os.path.join(OUTPUT_DIR, app_name)
    os.makedirs(out_dir, exist_ok=True)
    # 예시: 주요 DB, 로그만 추출 (실제 파일명은 앱에 맞게 수정)
    for fname in os.listdir(app_path):
        if fname.endswith(".db") or "log" in fname:
            src = os.path.join(app_path, fname)
            dst = os.path.join(out_dir, fname)
            shutil.copy2(src, dst)
    print(f"{app_name} 아티팩트 복사 완료")

def analyze_artifacts():
    summary = {}
    for app_dir in os.listdir(OUTPUT_DIR):
        files = os.listdir(os.path.join(OUTPUT_DIR, app_dir))
        summary[app_dir] = files
    with open(os.path.join(OUTPUT_DIR, "artifact_summary.json"), "w", encoding="utf-8") as fjson:
        json.dump(summary, fjson, indent=2)
    print("아티팩트 요약 저장 완료")

def main():
    print("[*] Wear OS 전체 자동화 파이프라인 시작")

    # Step 1. 암호화 이미지 추출
    if not adb_pull_image():
        return
    # Step 2. Node.js 복호화 실행
    if not decrypt_image_by_node():
        return
    # Step 3. 복호화 이미지 마운트 및 분석
    app_dirs = mount_image_and_extract_apps()
    for app in app_dirs:
        extract_artifacts_from_path(app)
    analyze_artifacts()
    print("[*] 전체 프로세스 완료")

if __name__ == "__main__":
    main()


[*] Wear OS 전체 자동화 파이프라인 시작
[0] 암호화 이미지 ADB로 추출 중...
ADB pull 실패: adb: error: failed to stat remote object '/data/user/0/com.android.example/...': No such file or directory

